# Static Architecture of a System

### Downloads

In [1]:
pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install plantuml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Imports

In [3]:
# Spacy Imports
import spacy 
from spacy.lang.en import English

# TextBlob Imports 
from textblob import TextBlob

# nltk Imports 
import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

#plantUML Imports 
import plantuml
from plantuml import PlantUML

# Other Imports 
import string
from os.path import abspath 
import collections
from collections import defaultdict

### nltk downloads

In [4]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

### PlantUML Code Generator 

In [5]:
def tokenAndPOS_Tags(sent): 
  nlp = English()
  token_sentence = nlp(sent)
  # Allows the tokenizing of special words, this way words containing all 
  # symbols outside of the ones outlined are not seperated
  # Eg: Inhibited_count>=2
  suffixes = nlp.Defaults.suffixes + [r"\-|\|\$",]
  suffix_regex = spacy.util.compile_suffix_regex(suffixes)
  nlp.tokenizer.suffix_search = suffix_regex.search

  doc = nlp(sent)
  token_sentence = []
  # Extract the token and store them in the token_sentence list
  for token in doc:
    token_sentence.append(token.text)

  # POS Tag the tokens 
  pos_tag_token = nltk.pos_tag(token_sentence)
  
  return token_sentence, pos_tag_token

In [6]:
# Noun Extraction functions 
def nounExtraction(tag_tokens):
  # Noun Extraction (only extracting Proper Nouns)
  nltk_nouns = []
  for index,tuple in enumerate(tag_tokens):
    if tuple[1] == 'NNP':
      nltk_nouns.append(tuple[0])
  return nltk_nouns

In [7]:
# List of Synonyms for consist and connect
# This is used oppose the stemming as certain Proper Nouns where being truncated 
def synonyms_lists(): 
  consist_synonyms = ['consist', 'consists','include','includes', 'comprise', 'comprises']
  connect_synonyms = ['connection','connections', 'connects', 'connect']
  input_synonyms = ['input', 'accepts', 'accept']

  return consist_synonyms, connect_synonyms, input_synonyms

In [8]:
# Editing or creating text file if it doesn't exists
# Creating the code for plantuml to generate a component based diagram

def staticPlantumlCode(text): 
  # Used to tokenize a sentence 
  doc = sent_tokenize(text)
  main_sub_component = []

  f = open('model_specs.txt','w')    
  # Iterate sentence-by-sentence then word-by-word 
  for sent in doc:
    # Call Tokenize and Stemming and POS Tag Function
    token_sentence, pos_tag_token = tokenAndPOS_Tags(sent)
            
    # Call Noun Extraction Function 
    # First noun is the main noun, the other nouns are its associations(inputs,
    # sub-components and outputs)
    nltk_nouns = nounExtraction(pos_tag_token)    
    
    # Used to increment the number of elements in the list of nouns
    i = 1
  
    # Call synonyms 
    consist_synonyms, connect_synonyms, input_synonyms = synonyms_lists()

    # create sets variable to check if consist synonyms are being used 
    t_sentence_set = set(token_sentence)
    consist_set = set(consist_synonyms)
    connect_set = set(connect_synonyms)
    input_set = set(input_synonyms)
    # If any synonym to consist exsists in the token list
    if (consist_set & t_sentence_set): 
      while(i < len(nltk_nouns)): 
        main_sub_component = nltk_nouns[1:]
        i = i + 1 
    
    # If any synonym to connect exsists in the token list
    if (connect_set & t_sentence_set):
      while(i < len(nltk_nouns)): 
          # Find the index of the first noun extracted in the main_sub_component
          # to prioritize order
          if(main_sub_component.index(nltk_nouns[0]) > 
             main_sub_component.index(nltk_nouns[i])):
            f.write(f"[{nltk_nouns[i]}]-[{nltk_nouns[0]}]\n")
          else: 
            f.write(f"[{nltk_nouns[0]}]-[{nltk_nouns[i]}]\n")             
          i = i+1

    # If any synonyms to input is in the token 
    if(input_set & t_sentence_set):
      # If input and clock is in the token 
      if "clock" in token_sentence: 
        while(i < len(nltk_nouns)): 
          f.write(f"{nltk_nouns[0]}<-up-({nltk_nouns[i]})\n")
          i = i+1
      # If input and boolean is in the token 
      elif "boolean" in token_sentence: 
        while(i < len(nltk_nouns)):    
            f.write(f"({nltk_nouns[i]}: boolean)->[{nltk_nouns[0]}]\n")
            i = i+1
      else: 
        while(i < len(nltk_nouns)):
          f.write(f"({nltk_nouns[i]})->[{nltk_nouns[0]}]\n")
          i = i+1
    # If output is in the token       
    if "output" in token_sentence: 
      while(i < len(nltk_nouns)): 
          f.write(f"{nltk_nouns[0]}->({nltk_nouns[i]})\n")
          i = i+1
  f.close()

### FGS System Example
Static Architecture

In [ ]:
# specification text
text = (" 1.	The FGS_System consists of four components: the Left_Side_FGS, an LR_Bus, and an RL_Bus, the Right_Side_FGS."
        " 2.	LR_Bus establishes connection between Left_Side_FGS and Right_Side_FGS."
        " 3.	The LR_Bus takes input from clock CLK2."
        " 4.	RL_Bus establishes connection between Left_Side_FGS and Right_Side_FGS."
        " 5.	The RL_Bus takes input from clock CLK4."
        " 6.	The Left_Side_FGS accepts as input a boolean value of Left_Transfer_Switch and Left_Primary_Side."
        " 8.	The Left_Side_FGS takes input from a synchronous clock CLK1."
        " 7.	The Right_Side_FGS accepts as input a boolean value of Right_Transfer_Switch and Right_Primary_Side."
        " 9.	The Right_Side_FGS takes input from a synchronous clock CLK3.")


In [12]:
sent_tokenize(text)

[' 1.',
 'The Active_Standby_System consists of four components: Side_1, Bus_12, Bus_21, and  Side_2.',
 '2.',
 'Bus_12 connects Side_1 with Side_2.',
 '3.',
 'Bus_12 takes input from clock CLK_12.',
 '4.',
 'Bus_12 takes input from Side_1_Status.',
 '5.',
 'Bus_21 connects Side_2 with Side_1.',
 '6.',
 'Bus_21 takes input from clock CLK_21.',
 '7.',
 'Side_1 accepts Manual_Selection, Side_1SubSystem_Status, and Side2SubSystem_Status as input.',
 '8.',
 'Side_1 accepts a input boolean value Side_1_Failed and Primary_Side as input.',
 '9.',
 'Side_1 takes input from clock CLK_1.',
 '10.',
 'Side_1 input signal is the Side_2_Status.',
 '11.',
 'Side_1 generates Side_1_Status as an output.',
 '13.',
 'Side_2 accepts Manual_Selection, Side_1SubSystem_Status, and Side2SubSystem_Status as input.',
 '14.',
 'Side_2 accepts a input boolean value, Side_2_Failed and Primary_Side as input.',
 '15.',
 'Side_2 takes Side_1_Status as input.',
 '16.',
 'Side_2 takes input from clock CLK_2.',
 '17.',


Creating System Diagram using PlantUML Code Generator for FGS Example

In [ ]:
staticPlantumlCode(text)
server = PlantUML(url='http://www.plantuml.com/plantuml/img/',
                          basic_auth={},
                          form_auth={}, http_opts={}, request_opts={})

server.processes_file(abspath('model_specs.txt'))


True

### Active Standby System Example
Static Architecture

In [9]:
text = (" 1.	The Active_Standby_System consists of four components: Side_1, Bus_12, Bus_21, and  Side_2."
        " 2.	Bus_12 connects Side_1 with Side_2."
        " 3.	Bus_12 takes input from clock CLK_12."
        " 4.	Bus_12 takes input from Side_1_Status."
        " 5.	Bus_21 connects Side_2 with Side_1."
        " 6.	Bus_21 takes input from clock CLK_21."
        " 7.	Side_1 accepts Manual_Selection, Side_1SubSystem_Status, and Side2SubSystem_Status as input." 
        " 8.	Side_1 accepts a input boolean value Side_1_Failed and Primary_Side as input."
        " 9.	Side_1 takes input from clock CLK_1."
        " 10.	Side_1 input signal is the Side_2_Status."
        " 11.	Side_1 generates Side_1_Status as an output."
        " 13.	Side_2 accepts Manual_Selection, Side_1SubSystem_Status, and Side2SubSystem_Status as input."
        " 14.	Side_2 accepts a input boolean value, Side_2_Failed and Primary_Side as input."
        " 15.	Side_2 takes Side_1_Status as input."
        " 16.	Side_2 takes input from clock CLK_2."
        " 17.	Side_2 generates Side_2_Status as an output.")


In [10]:
sent_tokenize(text)

[' 1.',
 'The Active_Standby_System consists of four components: Side_1, Bus_12, Bus_21, and  Side_2.',
 '2.',
 'Bus_12 connects Side_1 with Side_2.',
 '3.',
 'Bus_12 takes input from clock CLK_12.',
 '4.',
 'Bus_12 takes input from Side_1_Status.',
 '5.',
 'Bus_21 connects Side_2 with Side_1.',
 '6.',
 'Bus_21 takes input from clock CLK_21.',
 '7.',
 'Side_1 accepts Manual_Selection, Side_1SubSystem_Status, and Side2SubSystem_Status as input.',
 '8.',
 'Side_1 accepts a input boolean value Side_1_Failed and Primary_Side as input.',
 '9.',
 'Side_1 takes input from clock CLK_1.',
 '10.',
 'Side_1 input signal is the Side_2_Status.',
 '11.',
 'Side_1 generates Side_1_Status as an output.',
 '13.',
 'Side_2 accepts Manual_Selection, Side_1SubSystem_Status, and Side2SubSystem_Status as input.',
 '14.',
 'Side_2 accepts a input boolean value, Side_2_Failed and Primary_Side as input.',
 '15.',
 'Side_2 takes Side_1_Status as input.',
 '16.',
 'Side_2 takes input from clock CLK_2.',
 '17.',


Creating System Diagram using PlantUML Code Generator

In [11]:
staticPlantumlCode(text)
server = PlantUML(url='http://www.plantuml.com/plantuml/img/',
                          basic_auth={},
                          form_auth={}, http_opts={}, request_opts={})

server.processes_file(abspath('model_specs.txt'))


True